<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_2_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 2.3.2: Power BI (conexión y visualizaciones)

## 1. ¿Qué es Power BI y por qué dominarlo?

**Power BI** es la herramienta de Business Intelligence de Microsoft que transforma datos en **dashboards interactivos** sin escribir código. Es el estándar en el ecosistema corporativo por tres razones:

- **Omnipresencia:** Más del 50% de Fortune 500 lo usa para reporting.
- **Accesibilidad:** Curva de aprendizaje suave para no programadores.
- **Ecosistema:** Integración perfecta con Excel, Azure, SQL Server y Office 365.

> **Clave:** Power BI democratiza los datos. Lo que antes requería un equipo de BI, ahora lo hace un analista en una tarde.

**Dato curioso:** Power BI nació en 2013 como complemento de Excel. Hoy tiene +200,000 organizaciones activas y es el principal competidor de Tableau.

---

## 2. Arquitectura: El triángulo de Power BI

Power BI funciona como un ecosistema integrado:

| Componente | Función | ¿Cuándo usarlo? |
|------------|---------|-----------------|
| **Power BI Desktop** | Aplicación para diseñar reportes | Desarrollo local, creación de modelos (gratis) |
| **Power BI Service** | Plataforma cloud para compartir | Publicación, colaboración, actualización automática |
| **Power BI Mobile** | App iOS/Android | Consultas en movimiento, alertas push |

**Flujo típico:**
```
1. DESKTOP          2. SERVICE         3. MOBILE
   ↓                   ↓                  ↓
Conectas datos → Publicas en nube → CEO ve KPIs
Creas modelos      Programas refresh    desde iPhone
Diseñas dashboards Compartes con equipo Recibe alertas
```

> **Analogía:** Desktop es tu cocina, Service es el restaurante, Mobile es el delivery.

---

## 3. Conexión a datos: Tu superpoder

Power BI se conecta a **más de 150 fuentes**. Estas son las críticas:

### 3.1 Tipos de conexión

| Tipo | Ejemplos | Ventaja | Limitación |
|------|----------|---------|------------|
| **Archivos** | Excel, CSV, JSON | Rápido, sin configuración | Actualización manual |
| **Bases de datos** | SQL Server, MySQL, PostgreSQL | Datos frescos, escalables | Requiere credenciales |
| **Cloud** | Azure, Snowflake, BigQuery | Automático, Big Data | Coste, latencia |
| **Servicios web** | Salesforce, Google Analytics, APIs | Datos en tiempo real | Configuración compleja |

### 3.2 Ejemplo: Conectar SQL Server

**Escenario:** Tienda online con transacciones en base de datos.

**Pasos:**
1. Power BI Desktop → **Obtener datos** → "SQL Server"
2. **Servidor:** `localhost` | **Base de datos:** `VentasDB`
3. Elegir modo:
   - **Importar:** Copia datos (dashboard rápido, datos estáticos)
   - **DirectQuery:** Consulta en vivo (siempre actualizado, más lento)
4. Seleccionar tabla `Transacciones` → **Transformar datos**

**¿Importar o DirectQuery?**

| Criterio | Importar ✅ | DirectQuery ✅ |
|----------|-------------|----------------|
| **Velocidad dashboard** | Muy rápido | Más lento |
| **Frescura datos** | Hasta próxima actualización | Tiempo real |
| **Tamaño datos** | Límite 1GB (versión free) | Sin límite |
| **Carga servidor** | Baja (una vez) | Alta (cada consulta) |
| **Uso típico** | Análisis histórico | Dashboards operativos |

> **Tip:** Para archivos +1GB, usa **DirectQuery** o divide en múltiples archivos por mes/año.

---

## 4. Power Query: El laboratorio de limpieza

Antes de visualizar, necesitas datos limpios. **Power Query** es el ETL visual de Power BI:

### 4.1 Transformaciones comunes

| Operación | Descripción | Ejemplo |
|-----------|-------------|---------|
| **Quitar columnas** | Eliminar innecesarias | Borrar `ID_interno` |
| **Filtrar filas** | Eliminar registros | Ventas > 0, sin nulos |
| **Reemplazar valores** | Corregir inconsistencias | "M" → "Madrid" |
| **Dividir columna** | Separar datos compuestos | "Apellido, Nombre" → 2 columnas |
| **Combinar** | JOIN de tablas | Ventas + Productos |
| **Agrupar** | Agregaciones | Total ventas por región |

### 4.2 Ejemplo real: E-commerce con datos sucios

**Problema:** CSV de 50,000 ventas con:
- Fechas en formato texto: "15-ene-2024"
- Precios con símbolo: "€1,200"
- Valores nulos en "Ciudad"
- Registros duplicados

**Solución en Power Query (5 minutos):**
```plaintext
1. Cambiar tipo de "Fecha" → Date
2. Reemplazar "€" y "," en "Precio" → Tipo Number
3. Rellenar hacia abajo "Ciudad" (Fill Down)
4. Quitar filas duplicadas
5. Crear columna: Venta_Neta = [Precio] * [Cantidad]
6. Cerrar y aplicar
```

**Equivalente en Python (para comparar):**
```python
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d-%b-%Y')
df['Precio'] = df['Precio'].str.replace('€','').str.replace(',','').astype(float)
df['Ciudad'].fillna(method='ffill', inplace=True)
df = df.drop_duplicates()
df['Venta_Neta'] = df['Precio'] * df['Cantidad']
```

> **Ventaja Power Query:** Visual, sin código, reproducible automáticamente.

---

## 5. Modelo de datos: La arquitectura invisible

Power BI usa un **modelo relacional**. Las tablas se conectan mediante **relaciones** (como SQL JOINs).

### 5.1 Tipos de relaciones

| Tipo | Cardinalidad | Ejemplo | Frecuencia |
|------|--------------|---------|------------|
| **1:N** | Uno a Muchos | 1 Cliente → N Pedidos | 90% de casos |
| **N:1** | Muchos a Uno | N Productos → 1 Categoría | Inversa de 1:N |
| **1:1** | Uno a Uno | 1 Empleado → 1 Nómina | Raro (<5%) |
| **N:N** | Muchos a Muchos | N Estudiantes ↔ N Cursos | Evitar si es posible |

### 5.2 Esquema estrella (best practice)

```plaintext
         Dim_Productos       Dim_Clientes
              ↓                    ↓
         [ProductoID]         [ClienteID]
              ↓                    ↓
         ┌────────────────────────────┐
         │   Fact_Ventas              │
         │                            │
         │ [ProductoID] [ClienteID]   │
         │ [FechaID] [Importe]        │
         └─────────────┬──────────────┘
                       ↓
                  [FechaID]
                       ↓
               Dim_Calendario
```

**Elementos clave:**
- **Fact table:** Transacciones (ventas, pedidos) con métricas numéricas
- **Dimension tables:** Maestros (productos, clientes, fechas)
- **Tabla Calendario:** CRÍTICA para análisis temporal (veremos en DAX)

### 5.3 Buenas prácticas

✅ **Hacer:**
- Siempre crear tabla **Calendario** con todas las fechas
- Separar hechos (transacciones) de dimensiones (maestros)
- Relaciones **unidireccionales** (evita bidireccionales)
- Nombres claros: `Ventas[ClienteID]` → `Clientes[ID]`

❌ **Evitar:**
- Relaciones bidireccionales (causan ambigüedad)
- Columnas calculadas pesadas (usa medidas DAX)
- Importar todo sin filtrar (enlentece el modelo)

---

## 6. DAX: El lenguaje de las métricas

**DAX (Data Analysis Expressions)** es el lenguaje de fórmulas de Power BI. Es como Excel con esteroides.

### 6.1 Medidas vs Columnas calculadas

| Aspecto | Medida | Columna calculada |
|---------|--------|-------------------|
| **Cuándo se calcula** | En tiempo de consulta | Al cargar/actualizar datos |
| **Uso** | KPIs, agregaciones dinámicas | Segmentación, filtros |
| **Rendimiento** | ⚡ Eficiente | 🐌 Consume memoria |
| **Contexto** | Filtra según visual | Fijo en cada fila |
| **Ejemplo** | `Total Ventas = SUM(Ventas[Importe])` | `Margen = [Precio] - [Coste]` |

> **Regla de oro:** Usa **medidas** siempre que puedas. Solo usa columnas calculadas para categorías o segmentaciones.

### 6.2 Fórmulas DAX esenciales

```dax
// 1. Total básico
Total Ventas = SUM(Ventas[Importe])

// 2. Ventas año anterior (time intelligence)
Ventas Año Anterior =
CALCULATE(
    [Total Ventas],
    SAMEPERIODLASTYEAR(Calendario[Fecha])
)

// 3. Crecimiento % interanual
Crecimiento YoY =
DIVIDE(
    [Total Ventas] - [Ventas Año Anterior],
    [Ventas Año Anterior],
    0  // valor si división por cero
)

// 4. Promedio móvil 7 días
Promedio 7d =
CALCULATE(
    AVERAGE(Ventas[Importe]),
    DATESINPERIOD(
        Calendario[Fecha],
        LASTDATE(Calendario[Fecha]),
        -7,
        DAY
    )
)

// 5. Top N dinámico
Top 5 Productos =
CALCULATE(
    [Total Ventas],
    TOPN(5, ALL(Productos[Nombre]), [Total Ventas], DESC)
)

// 6. Condicional complejo
Estado Venta =
SWITCH(
    TRUE(),
    [Total Ventas] >= 10000, "Excelente",
    [Total Ventas] >= 5000, "Bueno",
    [Total Ventas] >= 1000, "Regular",
    "Bajo"
)
```

### 6.3 Crear tabla Calendario (indispensable)

```dax
Calendario =
CALENDAR(
    DATE(2023, 1, 1),
    DATE(2025, 12, 31)
)
```

**Luego añade columnas calculadas:**
```dax
Año = YEAR(Calendario[Fecha])
Mes = FORMAT(Calendario[Fecha], "MMM")
Trimestre = "Q" & FORMAT(Calendario[Fecha], "Q")
DiaSemana = FORMAT(Calendario[Fecha], "dddd")
EsFinDeSemana = IF(WEEKDAY(Calendario[Fecha]) IN {1, 7}, "Sí", "No")
```

> **¿Por qué es crítica?** Funciones DAX de tiempo (YTD, QTD, YoY) requieren una tabla continua de fechas.

---

## 7. Visualizaciones: Del dato al insight

Power BI ofrece **50+ tipos de visualizaciones**. Domina estos esenciales:

### 7.1 Catálogo de visualizaciones

| Visual | Cuándo usarlo | Ejemplo | Limitación |
|--------|---------------|---------|------------|
| **Gráfico de barras/columnas** | Comparar categorías | Top 10 productos | Máximo 20 categorías |
| **Línea** | Tendencias temporales | Ventas mensuales 2023-2025 | Evita >5 líneas |
| **Tarjeta** | KPI único destacado | Ventas hoy: €45.2K | Solo 1 número |
| **Tarjeta múltiple** | 2-4 KPIs relacionados | Ventas, Margen, Clientes | Máximo 4 métricas |
| **Tabla/Matriz** | Detalles, drill-down | Ventas por región y mes | Puede ser abrumadora |
| **Mapa** | Datos geográficos | Ventas por país | Requiere lat/long |
| **Gráfico circular** | Proporciones (úsalo poco) | % ventas por categoría | Máximo 5 categorías |
| **Embudo** | Proceso con etapas | Visita → Carrito → Compra | Debe ser secuencial |
| **Medidor** | Progreso hacia objetivo | 75% del objetivo anual | Necesita target |
| **Treemap** | Jerarquías (parte del todo) | Market share por producto | Difícil con >15 items |
| **Gráfico de dispersión** | Correlación entre 2 variables | Precio vs Demanda | Requiere muchos puntos |
| **Cascada** | Cambios acumulativos | De €100K a €150K | Mejor para 5-10 pasos |

### 7.2 Ejemplo: Dashboard de ventas e-commerce

**Caso: "TechStore" - Tienda online de electrónica**

**Layout del dashboard:**
```plaintext
┌──────────────────────────────────────────────────────┐
│ 💰 Ventas Hoy      📊 Pedidos      👥 Clientes       │
│    €23,450 (+12%)     342 (+8%)       127 (+5%)      │
├───────────────────────┬──────────────────────────────┤
│  📈 Tendencia 30d     │  🗺️ Mapa: Ventas por ciudad │
│  (Gráfico de líneas)  │  (Tamaño = €, Color = Δ%)   │
├───────────────────────┴──────────────────────────────┤
│  📊 Top 5 Productos (Barras horizontales)            │
│  1. iPhone 15 Pro ████████████ €8,450               │
│  2. MacBook Air   ██████████ €6,200                 │
├──────────────────────────────────────────────────────┤
│  🎯 Embudo de conversión: Web → Carrito → Compra    │
│     10,000 → 1,200 → 342  (Tasa: 3.4%)              │
└──────────────────────────────────────────────────────┘
Filtros: [Categoría ▾] [Ciudad ▾] [Fecha: Últimos 7d ▾]
```

**Visualizaciones usadas:**
1. **Tarjetas:** Ventas, Pedidos, Clientes (con Δ% vs ayer)
2. **Gráfico de líneas:** Evolución últimos 30 días
3. **Mapa:** Geolocalización de ventas
4. **Barras horizontales:** Ranking de productos
5. **Embudo:** Conversión del funnel

**Impacto:** El CEO detecta caída de conversión en Madrid (de 3.8% a 2.1%). Investiga y descubre problema de pago con tarjetas. **Pérdida evitada:** €15K/semana.

---

## 8. Interactividad: Haz que los dashboards hablen

Power BI no es estático. Los usuarios exploran los datos en tiempo real:

### 8.1 Funciones interactivas

| Función | Descripción | Ejemplo |
|---------|-------------|---------|
| **Segmentadores (Slicers)** | Filtros visuales interactivos | Dropdown de categorías |
| **Drill-down/up** | Navegar jerarquías | Año → Trimestre → Mes → Día |
| **Cross-filtering** | Un visual filtra automáticamente otros | Click en "Madrid" → Todo se filtra |
| **Tooltips personalizados** | Información extra al pasar ratón | Mostrar gráfico de tendencia en tooltip |
| **Botones y marcadores** | Navegación entre páginas | Botón "Ver análisis regional" |
| **Drill-through** | Ir a página de detalle | Click en producto → Página de ese producto |

### 8.2 Ejemplo de drill-down temporal

**Configuración:**
1. Crear jerarquía: `Fecha > Año > Trimestre > Mes`
2. Arrastrar jerarquía al eje X de gráfico de líneas
3. Activar drill-down (icono de flecha hacia abajo)

**Experiencia del usuario:**
```plaintext
Vista 1: Año        Vista 2: Trimestre    Vista 3: Mes
2023 ──────────┐    Q1 ─────────┐        Enero ─────┐
2024 ──────────│    Q2 ─────────│        Febrero ───│
2025 ─▶────────┘    Q3 ─────────│        Marzo ─▶───┘
                    Q4 ─▶───────┘
```

> **Tip:** Usa **tooltips personalizados** para mostrar gráficos de tendencia sin saturar el dashboard principal.

---

## 9. Publicación y colaboración

### 9.1 De Desktop a Service

**Flujo de publicación:**
1. Power BI Desktop → **Publicar** (arriba derecha)
2. Seleccionar **workspace** (Mi área de trabajo / Compartido)
3. Configurar **actualización automática**:
   - Diaria (8:00 AM)
   - Semanal (lunes 6:00 AM)
   - Mensual (día 1 a las 2:00 AM)
4. Compartir con usuarios

### 9.2 Tipos de compartición

| Método | Audiencia | Control | Costo |
|--------|-----------|---------|-------|
| **Compartir enlace** | Usuarios específicos (con licencia) | Pueden explorar | Requiere Pro |
| **App** | Equipos completos | Vista de solo lectura | Requiere Pro |
| **Insertar en web** | Público (⚠️ cuidado) | Datos 100% públicos | Gratis |
| **Exportar PDF/PowerPoint** | Presentaciones | Estático, snapshot | Gratis |

### 9.3 Row Level Security (RLS)

**Problema:** No todos deben ver todos los datos.

**Solución:** Configurar RLS para que cada usuario vea solo sus datos.

**Ejemplo:**
```dax
// En tabla Ventas, crear rol "Vendedor"
[Email_Vendedor] = USERPRINCIPALNAME()
```

**Resultado:** Juan solo ve ventas de Juan, María solo ve ventas de María.

---

## 10. Casos de uso reales

### Caso 1: Retail - Inditex (Zara)
**Problema:** 2,000 tiendas, datos dispersos en múltiples sistemas.  
**Solución Power BI:**
- Conexión a SAP (ventas), Excel (inventario), APIs (clima)
- Dashboard con actualización cada 15 minutos
- Métricas: Ventas/m², rotación stock, margen por prenda

**Visualizaciones clave:**
- Mapa de España con ventas por tienda
- Heatmap: Categoría × Tienda
- Gráfico de cascada: Stock inicial → Ventas → Reposición → Stock final

**Impacto:** Detectan tendencias 3 días antes que antes. Optimizan reposición. **Ahorro:** €15M anuales en exceso de stock.

### Caso 2: Startup Fintech - "PayFast"
**Problema:** Inversores piden KPIs semanales (MRR, churn, LTV, CAC).  
**Solución Power BI:**
- Conexión directa a PostgreSQL
- Dashboard con 8 KPIs principales
- Automatización total: cada lunes a las 9:00 AM

**KPIs en dashboard:**
```plaintext
MRR: €125K (+18% MoM)    Churn: 2.3% (-0.5pp)
Clientes activos: 3,450   LTV/CAC: 4.2x
```

**Impacto:** De 1 día de trabajo manual → 2 clicks. Inversores impresionados. **Ronda conseguida:** €5M Serie A.

### Caso 3: Logística - Empresa de paquetería
**Problema:** Entregas retrasadas, falta de visibilidad en rutas.  
**Solución Power BI:**
- GPS de camiones vía API en tiempo real
- Dashboard operativo para despachadores
- Alertas automáticas si retraso >30 min

**Dashboard muestra:**
- Mapa con posición de cada camión
- Progreso de ruta (% completado)
- Predicción de llegada vs horario comprometido

**Impacto:** Entregas a tiempo de 82% → 94%. NPS +25 puntos.

---

## 11. Errores comunes y cómo evitarlos

| Error | Consecuencia | Solución |
|-------|--------------|----------|
| Importar todo sin filtrar | Modelo lento, +2GB | Filtra en Power Query, solo últimos 2 años |
| Relaciones mal configuradas | Números incorrectos, duplicados | Verifica cardinalidad (1:N es la correcta 90%) |
| Demasiados visuales por página | Dashboard lento, confuso | Máximo 6-8 visuales por página |
| No usar tabla Calendario | Time intelligence no funciona | Crear con `CALENDAR()` |
| Colores sin criterio | Ilegible, poco profesional | Paleta corporativa, máximo 5 colores |
| Sin contexto en números | "€120K" ¿es bueno? | Añade comparativa: "€120K (+15% vs Q3)" |
| Usar gráfico circular con 12 categorías | Imposible leer | Barras horizontales o treemap |
| Columnas calculadas en vez de medidas | Modelo pesado y lento | Usa medidas DAX cuando sea posible |

---

## 12. Buenas prácticas profesionales

### 12.1 Diseño visual

✅ **Hacer:**
- **Jerarquía visual:** KPIs destacados arriba-izquierda
- **Regla de 3:** Máximo 3 colores principales
- **Whitespace:** Respira, no llenes cada píxel
- **Mobile-first:** Diseña pensando en tablets/móviles
- **Títulos descriptivos:** "Ventas caen 15% en Q3" > "Gráfico de ventas"

❌ **Evitar:**
- Más de 8 visuales por página
- Fondos oscuros con texto claro (difícil de leer)
- Animaciones excesivas
- Gráficos 3D (distorsionan percepción)

### 12.2 Rendimiento

✅ **Optimizar:**
- Usar medidas en vez de columnas calculadas
- Filtrar datos en Power Query, no en visuales
- Evitar relaciones bidireccionales
- Deshabilitar actualizaciones automáticas en desarrollo

❌ **Evitar:**
- Importar columnas que no usarás
- Crear 50 medidas "por si acaso"
- Relaciones N:N sin tabla puente

### 12.3 Narrativa de datos

**Regla de oro:** Si tu jefe no entiende el dashboard en 30 segundos, rediseña.

**Estructura recomendada:**
```plaintext
Página 1: RESUMEN EJECUTIVO
- 3-4 KPIs principales con variación %
- 1 gráfico de tendencia general
- 1 visual geográfico o de categorías

Página 2: ANÁLISIS PROFUNDO
- Drill-down por categorías
- Comparativas temporales (YoY, MoM)
- Tablas de detalle

Página 3: ACCIONES
- Alertas o puntos de atención (rojo)
- Oportunidades (verde)
- Recomendaciones basadas en datos
```

---

## 13. Power BI vs la competencia

| Criterio | Power BI ✅ | Tableau | Looker Studio | Python (Plotly) |
|----------|------------|---------|---------------|-----------------|
| **Curva de aprendizaje** | Media | Media-alta | Baja | Alta |
| **Coste mensual** | €10-20/user | €70/user | Gratis | Gratis |
| **Ecosistema Microsoft** | Perfecto | Regular | Débil | N/A |
| **Visualizaciones avanzadas** | Bueno | Excelente | Básico | Ilimitado |
| **Modelos complejos** | Excelente (DAX) | Bueno | Limitado | Excelente |
| **Colaboración** | Muy buena | Excelente | Buena | Pobre |
| **Autonomía del usuario** | Alta | Alta | Media | Baja (requiere dev) |
| **Primer dashboard** | 2 horas | 3 horas | 1 hora | 8 horas |
| **Integración con SQL** | Nativa | Nativa | Regular | Excelente |

**Usa Power BI cuando:**
- Tu empresa está en ecosistema Microsoft
- Necesitas dashboards empresariales rápido
- Usuarios finales no son técnicos
- Presupuesto moderado (€10-20/user/mes)
- Modelos de datos complejos con DAX

**Usa Tableau cuando:**
- Visualizaciones pixel-perfect son críticas
- Presupuesto no es problema
- Análisis exploratorio ad-hoc intensivo

**Usa Looker Studio cuando:**
- Presupuesto = €0
- Solo necesitas Google Analytics/Sheets
- Dashboards simples de marketing

**Usa Python cuando:**
- Control total del código es necesario
- Análisis estadístico avanzado
- Integración con ML pipelines

---

## 14. Ejercicio práctico: Tu primer dashboard

### Contexto
Eres analista en **ModaExpress**, cadena de moda con 50 tiendas en España.

### Objetivos
1. Identificar productos más rentables
2. Analizar rendimiento por tienda
3. Detectar estacionalidad en ventas

### Dataset (crear CSV: ventas_modaexpress.csv)
```csv
Fecha,Tienda,Producto,Categoria,Precio,Cantidad,Coste
2024-01-15,Madrid Centro,Camiseta,Ropa,25,12,10
2024-01-15,Barcelona Diagonal,Pantalón,Ropa,60,8,25
2024-01-16,Valencia Puerto,Zapatillas,Calzado,80,5,40
2024-01-17,Sevilla Nervión,Camiseta,Ropa,25,15,10
2024-02-10,Madrid Centro,Abrigo,Ropa,120,3,60
```

### Pasos del ejercicio (45 minutos)
1. **Conectar datos**
   - Importar CSV a Power BI Desktop
   
2. **Transformar en Power Query**
   - Crear columna: `Venta_Neta = [Precio] * [Cantidad]`
   - Crear columna: `Margen = [Venta_Neta] - ([Coste] * [Cantidad])`
   - Crear columna: `Margen_% = [Margen] / [Venta_Neta]`

3. **Crear tabla Calendario**
   ```dax
   Calendario = CALENDAR(DATE(2024,1,1), DATE(2024,12,31))
   ```

4. **Crear medidas DAX**
   ```dax
   Total Ventas = SUM(Ventas[Venta_Neta])
   Total Margen = SUM(Ventas[Margen])
   Productos Vendidos = SUM(Ventas[Cantidad])
   ```

5. **Diseñar dashboard con:**
   - 3 tarjetas: Total Ventas, Margen, Productos vendidos
   - Gráfico de barras: Ventas por Tienda
   - Gráfico de líneas: Tendencia mensual
   - Tabla: Top 10 productos por margen %
   - Slicer: Filtro de Categoría

6. **Bonus: Análisis avanzado**
   - Añadir mapa con ventas por ciudad
   - Crear medida: `Ticket Promedio = [Total Ventas] / [Productos Vendidos]`
   - Añadir tooltip personalizado en gráfico de barras

### Insights esperados
- ¿Qué tienda es más rentable?
- ¿Qué productos tienen mejor margen?
- ¿Hay estacionalidad en las ventas?
- ¿Qué categoría debería impulsar marketing?

---

## 15. Camino de aprendizaje

**Nivel 1 - Fundamentos (2 semanas)**
- ✅ Conectar Excel/CSV
- ✅ Transformaciones básicas en Power Query
- ✅ Crear gráficos (barras, líneas, tarjetas)
- ✅ Publicar en Power BI Service

**Nivel 2 - Intermedio (1 mes)**
- ✅ Dominar Power Query (joins, group by)
- ✅ DAX básico (SUM, AVERAGE, CALCULATE)
- ✅ Modelos relacionales (esquema estrella)
- ✅ Dashboards interactivos con slicers

**Nivel 3 - Avanzado (3 meses)**
- ✅ DAX avanzado (time intelligence, iteradores)
- ✅ Optimización de rendimiento
- ✅ Row Level Security
- ✅ Parámetros y escenarios what-if

**Nivel 4 - Experto (6+ meses)**
- ✅ Arquitectura de soluciones enterprise
- ✅ Integración con APIs (Python/R scripts)
- ✅ Certificación PL-300
- ✅ Power BI Embedded para aplicaciones

---

## 16. Resumen

- **Power BI** es la herramienta estándar de BI en el ecosistema Microsoft
- **Power Query** limpia y transforma datos sin código
- **DAX** crea métricas inteligentes y análisis temporal
- **Modelos relacionales** son la base de dashboards escalables
- **Interactividad** convierte datos estáticos en herramientas de exploración
- **Service** permite colaboración y automatización en la nube

> **Meta final:** Crear dashboards que respondan preguntas antes de que las hagan.

**Tres cosas que debes recordar:**
1. **Medidas > Columnas calculadas** para rendimiento
2. **Tabla Calendario** es obligatoria para time intelligence
3. **Menos es más:** 6 visuales bien diseñados > 15 desordenados

---

## 17. Referencias

### Vídeos
- [Power BI Full Course - Learn Power BI in 4 Hours](https://youtu.be/77v83_VJ4MQ)
- [Power BI Tutorial From Beginner to Pro](https://youtu.be/AGrl-H87pRU)
- [Power BI DAX Tutorial for Beginners](https://youtu.be/klQAZLr5vxA)
- [How to Design Beautiful Dashboards in Power BI](https://youtu.be/7vIEF8vCMQ8)

### Lecturas
- [Microsoft Learn - Power BI](https://learn.microsoft.com/es-es/power-bi/)
- [SQLBI - DAX Patterns](https://www.daxpatterns.com/)
- [Power BI Best Practices](https://learn.microsoft.com/en-us/power-bi/guidance/power-bi-optimization)
- [DAX Guide - Referencia completa](https://dax.guide/)

### Datasets de práctica
- [Microsoft Sample Datasets](https://learn.microsoft.com/power-bi/create-reports/sample-datasets)
- [Kaggle - Superstore Sales](https://www.kaggle.com/datasets/vivek468/superstore-dataset-final)
- [World Bank Open Data](https://data.worldbank.org/)
- [European Open Data Portal](https://data.europa.eu/)

### Certificaciones
- **PL-300:** Microsoft Power BI Data Analyst (oficial)
- [Ruta de certificación Microsoft](https://learn.microsoft.com/certifications/power-bi-data-analyst-associate/)

---

**¡Ahora te toca practicar! Descarga Power BI Desktop (gratis) y crea tu primer dashboard con el ejercicio de ModaExpress. La mejor forma de aprender Power BI es ensuciándote las manos con datos reales. 🚀📊**